# Louvain Performance Benchmarking

This notebook benchmarks performance improvement of running the Louvain clustering algorithm within cuGraph against NetworkX. The test is run over eight test networks (graphs) and then results plotted.  
<p><p>


#### Notebook Credits

    Original Authors: Bradley Rees
    Last Edit: 08/06/2019


#### Test Environment

    RAPIDS Versions: 0.9.0

    Test Hardware:
    GV100 32G, CUDA 10,0
    Intel(R) Core(TM) CPU i7-7800X @ 3.50GHz
    32GB system memory



#### Updates
- moved loading ploting libraries to front so that dependencies can be checked before running algorithms
- added edge values 
- changed timing to including Graph creation for both cuGraph and NetworkX.  This will better represent end-to-end times



#### Dependencies
- RAPIDS cuDF and cuGraph version 0.6.0 
- NetworkX 
- Matplotlib 
- Scipy 
- data prep script run



#### Note: Comparison against published results


The cuGraph blog post included performance numbers that were collected over a year ago.  For the test graphs, int32 values are now used.  That improves GPUs performance.  Additionally, the initial benchamrks were measured on a P100 GPU. 

This test only comparse the modularity scores and a success is if the scores are within 15% of each other.  That comparison is done by adjusting the NetworkX modularity score and then verifying that the cuGraph score is higher.

cuGraph did a full validation of NetworkX results against cuGraph results.  That included cross-validation of every cluster.  That test is very slow and not included here

In [2]:
# Import needed libraries
import time
import cugraph
import cudf
import os

In [3]:
# NetworkX libraries
try: 
    import community
except ModuleNotFoundError:
    os.system('pip install python-louvain')
    import community
import networkx as nx
from scipy.io import mmread

In [4]:
# Loading plotting libraries
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

In [5]:
!bash dataPrep.sh

mkdir: cannot create directory 'data': File exists
--2019-11-01 20:49:03--  https://sparse.tamu.edu/MM/DIMACS10/preferentialAttachment.tar.gz
Resolving sparse.tamu.edu (sparse.tamu.edu)... 128.194.136.136
Connecting to sparse.tamu.edu (sparse.tamu.edu)|128.194.136.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2027782 (1.9M) [application/x-gzip]
Saving to: 'preferentialAttachment.tar.gz'

preferentialAttachm 100%[===================>]   1.93M  3.48MB/s    in 0.6s    

2019-11-01 20:49:04 (3.48 MB/s) - 'preferentialAttachment.tar.gz' saved [2027782/2027782]

--2019-11-01 20:49:04--  https://sparse.tamu.edu/MM/DIMACS10/caidaRouterLevel.tar.gz
Resolving sparse.tamu.edu (sparse.tamu.edu)... 128.194.136.136
Connecting to sparse.tamu.edu (sparse.tamu.edu)|128.194.136.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2418742 (2.3M) [application/x-gzip]
Saving to: 'caidaRouterLevel.tar.gz'

caidaRouterLevel.ta 100%[===================>

### Define the test data

In [6]:
# Test File
data = {
    'preferentialAttachment' : './data/preferentialAttachment.mtx',
    'caidaRouterLevel'       : './data/caidaRouterLevel.mtx',
    'coAuthorsDBLP'          : './data/coAuthorsDBLP.mtx',
    'dblp'                   : './data/dblp-2010.mtx',
    'citationCiteseer'       : './data/citationCiteseer.mtx',
    'coPapersDBLP'           : './data/coPapersDBLP.mtx',
    'coPapersCiteseer'       : './data/coPapersCiteseer.mtx',
    'as-Skitter'             : './data/as-Skitter.mtx'
}

### Define the testing functions

In [7]:
# Read in a dataset in MTX format 
def read_mtx_file(mm_file):
    print('Reading ' + str(mm_file) + '...')
    d = mmread(mm_file).asfptype()
    M = d.tocsr()
    
    if M is None:
        raise TypeError('Could not read the input graph')
    if M.shape[0] != M.shape[1]:
        raise TypeError('Shape is not square')
        
    return M

In [8]:
# Run the cuGraph Louvain analytic (using nvGRAPH function)
def cugraph_call(M):

    t1 = time.time()

    # data
    row_offsets = cudf.Series(M.indptr)
    col_indices = cudf.Series(M.indices)
    data        = cudf.Series(M.data)
    
    # create graph 
    G = cugraph.Graph()
    G.add_adj_list(row_offsets, col_indices, data)

    # cugraph Louvain Call
    print('  cuGraph Solving... ')
    df, mod = cugraph.louvain(G)   
    
    t2 = time.time() - t1
    return t2, mod


In [9]:
# Run the NetworkX Louvain analytic.  THis is done in two parts since the modularity score is not returned 
def networkx_call(M):
   
    t1 = time.time()

    # Directed NetworkX graph
    Gnx = nx.Graph(M)

    # Networkx 
    print('  NetworkX Solving... ')
    parts = community.best_partition(Gnx)
    
    # Calculating modularity scores for comparison 
    mod = community.modularity(parts, Gnx)   
    
    t2 = time.time() - t1
    
    return t2, mod

### Run the benchmarks

In [ ]:
# Loop through each test file and compute the speedup
perf  = []
names = []

for k,v in data.items():
    M = read_mtx_file(v)
    tr, modc = cugraph_call(M)
    tn, modx = networkx_call(M)
    
    speedUp = (tn / tr)
    names.append(k)
    perf.append(speedUp)
    
    mod_delta = (0.85 * modx)
    
    print(str(speedUp) + "x faster =>  cugraph " + str(tr) + " vs " + str(tn))
    print("Modularity =>  cugraph " + str(modc) + " should be greater than " + str(mod_delta))

Reading ./data/preferentialAttachment.mtx...
  cuGraph Solving... 
  NetworkX Solving... 
3509.4500202625027x faster =>  cugraph 0.8648371696472168 vs 3035.1028225421906
Modularity =>  cugraph 0.19461682219817675 should be greater than 0.21973558127621454
Reading ./data/caidaRouterLevel.mtx...
  cuGraph Solving... 
  NetworkX Solving... 
7076.7607431556x faster =>  cugraph 0.04834103584289551 vs 342.0979447364807
Modularity =>  cugraph 0.7872923202092253 should be greater than 0.7289947349239256
Reading ./data/coAuthorsDBLP.mtx...
  cuGraph Solving... 
  NetworkX Solving... 
11893.139026724633x faster =>  cugraph 0.06750750541687012 vs 802.8761472702026
Modularity =>  cugraph 0.7648739273488195 should be greater than 0.7026254024456955
Reading ./data/dblp-2010.mtx...
  cuGraph Solving... 
  NetworkX Solving... 
12969.744546806074x faster =>  cugraph 0.07826042175292969 vs 1015.0176782608032
Modularity =>  cugraph 0.7506256512679915 should be greater than 0.7450002914515801
Reading ./da

### plot the output

In [ ]:
%matplotlib inline

y_pos = np.arange(len(names))
 
plt.bar(y_pos, perf, align='center', alpha=0.5)
plt.xticks(y_pos, names)
plt.ylabel('Speed Up')
plt.title('Performance Speedup: cuGraph vs NetworkX')
plt.xticks(rotation=90) 
plt.show()